# Demo: Retrieving Mgnify tomato samples

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ebi-metagenomics/mgnipy/blob/main/docs/tutorial/demo-mgnify-tomatoes.ipynb)

In [ ]:
# uncomment below if colab
#!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple mgnipy
#!pip install asyncio

In this tutorial we demonstrate how mgnipy can be used to retrieve tomato rhizosphere metagenomics analyses (only sample metadata for now) available on MGnify. 

## Spying into the metadata using a `Mgnifier`
We can use a Mgnifier to get metadata for resources: 
- `studies` with option to filter by biome, pipeline ver, and search terms
- `samples` for a given study accession
- `analyses` for a given study accession
- `genomes` given its accession

In [ ]:
from mgnipy.V2 import StudiesMgnifier

you can either pass query parameters as dict to `params` or as kwargs. Please refer to [mgnify api docs](https://www.ebi.ac.uk/metagenomics/api/v2/) for the accepted kwargs for now or via attribute `Mgnifier.supported_kwargs`

In [ ]:
# init 
glass = StudiesMgnifier(
    biome_lineage="root:Host-associated:Plants:Rhizosphere",
    search="tomato",
    page_size=5
)

print(glass)

the mgnifier has been initiated only, no request made yet. 

Next we must plan or preview before carrying out the full request (of all page results) 

In [ ]:
glass.plan()

previewing is basically the same as planning but returns the first page results as a pandas.DataFrame

In [ ]:
glass.preview()

nice okay let's get the rest of the records (the 3 other pages)

In [ ]:
import asyncio
study_meta = await glass.get()

# check it out
display(study_meta)

The matching study accessions can be accessed via attribute `.accessions`

## Getting the samples from the studies

We can use `SamplesMgnifier` to get the sample accessions for those study accessioins. 

In [ ]:
from mgnipy.V2 import SamplesMgnifier

# init
by_study = {}
for acc in glass.accessions:
    lens = SamplesMgnifier(
        study_accession=acc
    )
    # verbose
    print(lens.plan())
    # get sample metadata 
    by_study[acc] = await lens.get()
    # spacing
    print("\n\n")

it wanting all in one df

In [ ]:
import pandas as pd

# concat
tomato_samples = pd.concat(by_study)

# check it out
tomato_samples.sample(5)

## Getting the analyses from the studies

We can use `AnalysesMgnifier` to get the analyses metadata for the study accessioins. 

In [ ]:
from mgnipy.V2 import AnalysesMgnifier

# init
analyses_by_study = {}
for acc in glass.accessions:
    lens = AnalysesMgnifier(
        study_accession=acc
    )
    # verbose
    print(lens.plan())
    # get analyses metadata, but here only first 2 pages for demo purposes 
    analyses_by_study[acc] = await lens.get(pages=[1,2])
    # spacing
    print("\n\n")

it wanting all in one df

In [ ]:
# currently no results

# # concat
# tomato_analyses = pd.concat(analyses_by_study)

# # check it out
# tomato_analyses.sample(5)